In [755]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [1313]:
df = pd.read_csv('default_data_all_variables.csv')
target = pd.read_csv('targets.csv')

In [1304]:
df = df.loc[:, ~df.columns.str.contains('HomeTeam|AwayTeam')]

In [1305]:
df[['home_win_odds','draw_odds','away_win_odds']] = target[['home_win_odds','draw_odds','away_win_odds']]

In [1306]:
target = target[['FTHG', 'FTAG']]

In [1311]:
X_train = df[:-1500]
y_train = target[:-1500]
X_val = df[-1500:-750]
y_val = target[-1500:-750]
X_test = df[-750:]
y_test = target[-750:]


In [1315]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0005)
model = Sequential()

# Input layer
model.add(Dense(256, activation='relu', input_shape=(182,)))  # 278 is the number of input features
# model.add(Dropout(0.1))
# Hidden layers
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))  # Optional, to prevent overfitting
# model.add(Dense(138, activation='relu'))
# model.add(Dropout(0.1))  # Optional, to prevent overfitting
model.add(Dense(48, activation='relu'))

# Output layer (with softmax to ensure sum-to-1 constraint)
model.add(Dense(2, activation='relu'))

# Compile the model
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
class_weights = {'home_win_odds': 1.0, 'draw_odds': 1, 'away_win_odds': 1.0}
# Train the model
history = model.fit(X_train, y_train, epochs=600, batch_size=1000, validation_data=(X_val, y_val), class_weight=class_weights, callbacks=[early_stopping])
# , callbacks=[early_stopping]
# Evaluate the model
results = model.evaluate(X_test, y_test)

Epoch 1/600


C:\Users\ciesl\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.2026 - mae: 1.0890 - val_loss: 1.5810 - val_mae: 0.9856
Epoch 2/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5409 - mae: 0.9377 - val_loss: 1.5147 - val_mae: 0.9418
Epoch 3/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4666 - mae: 0.9178 - val_loss: 1.4371 - val_mae: 0.9204
Epoch 4/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4294 - mae: 0.9123 - val_loss: 1.4046 - val_mae: 0.9056
Epoch 5/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3982 - mae: 0.8981 - val_loss: 1.3976 - val_mae: 0.9029
Epoch 6/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3706 - mae: 0.8951 - val_loss: 1.3930 - val_mae: 0.9089
Epoch 7/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3725 - mae: 0.8946 - val_loss: 1.3820 - val_mae: 0.9072
Epoch 8/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3491 - mae: 0.8908 - val_loss: 1.3785 - val_mae: 0.9160
Epoch 9/600
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3543 - mae: 0.

In [1316]:
results

[1.413450837135315, 0.9116207957267761]

In [1317]:
y_pred = model.predict(X_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [1318]:
def choose_value2(row, val_draw):
    if row['FTHG']-val_draw > row['FTAG']:
        return 'H'
    elif np.abs(row['FTHG']- row['FTAG']) <val_draw:
        return 'D'
    else:
        return 'A'

In [1323]:
trg = target[-750:]
trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)
trg.reset_index(inplace = True, drop = True)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\89127985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)


In [1325]:
trg

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds,result
0,3,3,0.566537,0.230440,0.203023,D
1,2,4,0.098600,0.175840,0.725560,A
2,2,1,0.719719,0.171116,0.109165,H
3,1,0,0.576157,0.235571,0.188272,H
4,1,0,0.518624,0.275270,0.206106,H
...,...,...,...,...,...,...
745,3,2,0.392395,0.292396,0.315209,H
746,2,0,0.486209,0.284551,0.229241,H
747,4,0,0.398873,0.273154,0.327973,H
748,1,0,0.533203,0.245773,0.221023,H


In [1327]:
res = pd.DataFrame(y_pred, columns = ['FTHG', 'FTAG'])
res['result'] = res.apply(choose_value2, val_draw = 0.2, axis=1)
matches = res['result'] == trg['result']

In [1337]:
over_under = pd.read_csv('over_under.csv')
over_under.reset_index(inplace = True, drop = True)

In [1343]:
res[['over_25', 'under_25']] = over_under[['Avg>2.5', 'Avg<2.5']]
res['total_g'] = res['FTHG']+ res['FTAG']
res['total_game'] = trg['FTHG']+ trg['FTAG']

In [1345]:
scr = res.loc[res['over_25']!=0]

In [1347]:
scr

,FTHG,FTAG,result,over_25,under_25,total_g,total_game
0,1.648709,1.112084,H,1.70,2.20,2.760793,6
1,0.724709,2.053399,A,1.54,2.49,2.778108,6
2,1.815382,0.846717,H,1.80,2.04,2.662098,3
3,1.558794,1.107114,H,1.46,2.66,2.665909,1
4,1.231003,0.993473,H,1.63,2.29,2.224476,1
...,...,...,...,...,...,...,...
745,0.894524,1.150219,A,1.50,2.48,2.044744,5
746,1.076726,0.752408,H,2.10,1.76,1.829134,2
747,1.407820,1.636510,A,1.68,2.22,3.044331,4
748,1.126478,0.928226,D,1.66,2.19,2.054704,1


In [1349]:
def money_won_goals(row):
    if row['total_g'] > 2.5:
        if row['total_game'] >2.5:
             return row['over_25']*0.88
        else:
            return 0
    else:
         if row['total_game'] <2.5:
             return row['under_25']*0.88
         else:
             return 0

In [1351]:
scr['stake'] = scr.apply(money_won_goals, axis=1)

In [1353]:
scr['stake'].sum()

750.4288

In [1355]:
percentage_compatibility = matches.mean() * 100
percentage_compatibility

49.733333333333334

In [1230]:
trg['res2'] = res['result']

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\512750724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['res2'] = res['result']


In [1232]:
trg

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds,result,res2
0,1,1,0.458753,0.266787,0.274460,D,H
1,1,1,0.628708,0.196715,0.174577,D,H
2,5,1,0.775290,0.136685,0.088025,H,H
3,2,1,0.481468,0.250448,0.268085,H,H
4,1,0,0.407562,0.271064,0.321374,H,D
...,...,...,...,...,...,...,...
495,3,2,0.392395,0.292396,0.315209,H,D
496,2,0,0.486209,0.284551,0.229241,H,H
497,4,0,0.398873,0.273154,0.327973,H,D
498,1,0,0.533203,0.245773,0.221023,H,H


In [1234]:
def money_won(row):
    if row['res2'] == row['result']:
        if row['res2'] == 'H':
            return 1/row['home_win_odds']*0.88/1.05
        elif row['res2'] == 'D':
            return 1/row['draw_odds']*0.88/1.05
        else:
             return 1/row['away_win_odds']*0.88/1.05
    else:
        return 0

In [1236]:
trg['money_won'] = trg.apply(money_won, axis=1)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\32439853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['money_won'] = trg.apply(money_won, axis=1)


In [1238]:
trg['money_won'].sum()

446.12666009462816

In [1240]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators = 200, max_depth = 30)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [1241]:
predictions = model.predict(X_test)

predictions

array([[2.1119072 , 0.6883068 ],
       [3.5416553 , 1.5363808 ],
       [3.1409314 , 0.7233491 ],
       [1.5026135 , 2.1081471 ],
       [2.1060932 , 0.7686245 ],
       [2.4062266 , 1.8194178 ],
       [0.6651306 , 2.2547727 ],
       [1.6988604 , 0.75649536],
       [2.10201   , 1.1307769 ],
       [1.4181185 , 1.5403137 ],
       [1.7123829 , 1.2747993 ],
       [2.035104  , 1.837677  ],
       [1.2699234 , 1.3760148 ],
       [1.7411501 , 1.1166574 ],
       [1.3313135 , 0.959963  ],
       [1.5420438 , 1.4895061 ],
       [2.3792255 , 0.9538844 ],
       [1.3065447 , 1.703225  ],
       [1.489832  , 1.4754621 ],
       [1.8995148 , 0.69232994],
       [2.1473293 , 1.2678943 ],
       [2.110924  , 0.83068085],
       [0.957619  , 0.9745591 ],
       [3.1441724 , 1.1852651 ],
       [1.8642128 , 1.0602038 ],
       [2.3792255 , 0.9538844 ],
       [1.4003555 , 0.66764003],
       [1.8642128 , 1.0602038 ],
       [3.1441724 , 1.1852651 ],
       [0.957619  , 0.9745591 ],
       [2.

In [1242]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)

1.02508966479823

In [1243]:
pred_df = pd.DataFrame(predictions, columns = ['home_win_odds', 'draw_odds', 'away_win_odds'])
pred_df[['FTHG', 'FTAG']] = trg[['FTHG', 'FTAG']]
pred_df['probability converted'] = pred_df.apply(choose_value, axis=1)
pred_df['probability converted'].sum()

ValueError: Shape of passed values is (500, 2), indices imply (500, 3)

In [ ]:
import matplotlib.pyplot as plt
feature_importances = model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for easier manipulation
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort by importance and get the top 20 features
top_20_features = feature_importances_df.sort_values(by='Importance', ascending=False).head(20)

# Plot the top 20 feature importances
plt.figure(figsize=(10, 6))
plt.barh(top_20_features['Feature'], top_20_features['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Top 20 Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

In [ ]:
target[-500:]

In [ ]:
trg = target[-500:]

In [ ]:
def choose_value(row):
    if row['FTHG'] > row['FTAG']:
        return 'H'
    elif row['FTHG'] == row['FTAG']:
        return 'D'
    else:
        return 'A'

# Apply the function to each row
trg['result'] = trg.apply(choose_value, axis=1)

In [ ]:
trg['result']

In [ ]:
# dodać zmienne związane z ovr 2.5 goli